# Tutorial: Permutation Testing using `acore`

In this notebook we will demonstrate how to use acore's permutation testing functions on metagenomics data collected by [Ju and colleagues (2018)](https://doi.org/10.1038/s41396-018-0277-8).

The samples were collected from wastewaster treatment plant inffluent and effluent.

## Downloading the dataset

The analysed samples can be downloaded via the Mgnify API.

In [ ]:
import requests 
from io import StringIO
import pandas as pd

enf_accession = "MGYS00005058"
inf_accession = "MGYS00005056"

enf_url = f'https://www.ebi.ac.uk/metagenomics/api/v1/studies/{enf_accession}/pipelines/4.1/file/ERP112696_GO_abundances_v4.1.tsv'
inf_url = f'https://www.ebi.ac.uk/metagenomics/api/v1/studies/{inf_accession}/pipelines/4.1/file/ERP111072_GO_abundances_v4.1.tsv'

# init dict to hold dataframes
data = {}
for url in [enf_url, inf_url]:
    print(f"Processing {url}")
    # download
    response = requests.get(url)
    if response.status_code == 200:
        # treat as file
        file_like = StringIO(response.content.decode('utf-8'))
        # read into pd df
        df = pd.read_csv(file_like, sep='\t')
        # to the dict
        data[url] = df
    else: 
        print(response)
        print(url, " download skipped")

# sanity check
data[enf_url].head()

Processing https://www.ebi.ac.uk/metagenomics/api/v1/studies/MGYS00005058/pipelines/4.1/file/ERP112696_GO_abundances_v4.1.tsv
Processing https://www.ebi.ac.uk/metagenomics/api/v1/studies/MGYS00005056/pipelines/4.1/file/ERP111072_GO_abundances_v4.1.tsv


,GO,description,category,ERR2985255,ERR2985256,ERR2985257,ERR2985258,ERR2985259,ERR2985260,ERR2985261,...,ERR2985269,ERR2985270,ERR2985271,ERR2985272,ERR2985273,ERR2985274,ERR2985275,ERR2985276,ERR2985277,ERR2985278
0,GO:0000001,mitochondrion inheritance,biological process,0,0,0,0,0,3,0,...,1,0,1,1,1,2,0,0,2,2
1,GO:0000002,mitochondrial genome maintenance,biological process,0,9,7,0,2,7,4,...,15,10,0,0,0,5,0,4,11,1
2,GO:0000012,single strand break repair,biological process,0,3,1,1,0,0,0,...,0,0,1,0,0,2,1,2,1,1
3,GO:0000015,phosphopyruvate hydratase complex,cellular component,477,384,500,188,226,232,301,...,203,432,152,341,319,520,290,349,270,292
4,GO:0000030,mannosyltransferase activity,molecular function,189,200,57,67,39,129,181,...,70,42,58,73,92,159,67,179,58,91


In [62]:
# preprocessing
from sklearn.preprocessing import FunctionTransformer
import numpy as np
from sklearn.pipeline import make_pipeline

coda = FunctionTransformer(lambda x: x / x.sum(axis=0), feature_names_out="one-to-one").set_output(transform="pandas")

def calc_clr(x):
    # replace zeros with small 
    new_x = np.where(x > 0, x, 1e-10)
    return np.log(new_x) - np.mean(np.log(new_x), axis=0)

clr = FunctionTransformer(calc_clr, feature_names_out="one-to-one").set_output(transform="pandas")

def coda_clr(df):
    pipe = make_pipeline(coda, clr).set_output(transform="pandas")
    transformed = pipe.fit_transform(df.select_dtypes(include='number'))
    return pd.concat([df.select_dtypes(include='object'), transformed], axis=1)

clr_enf = coda_clr(data[enf_url])
clr_inf = coda_clr(data[inf_url])

For this demo we will only look at [go term GO:0017001](https://www.ebi.ac.uk/QuickGO/term/GO:0017001)

It's expected that antibiotic catabolic processes to be higher in influent vs efluent

In [130]:
term = 'GO:0017001'

inf_go = clr_inf.query("GO == @term").drop(columns=['GO', 'description', 'category']).T.reset_index()
enf_go = clr_enf.query("GO == @term").drop(columns=['GO', 'description', 'category']).T.reset_index()

inf_go.columns = ['sample', 'abundance']
enf_go.columns = ['sample', 'abundance']

The samples are paired. We need the metadata for the samples which we can also download using the MGnify API.

In [131]:
# getting sampling and analysis metadata
metadata = {}
for accession in [enf_accession, inf_accession]:
    # prep urls
    url_sam = f'https://www.ebi.ac.uk/metagenomics/api/v1/studies/{accession}/samples'
    url_ana = f"https://www.ebi.ac.uk/metagenomics/api/v1/analyses?study_accession={accession}"

    print(f"Processing {accession}")
    # download
    sam_response = requests.get(url_sam)
    if sam_response.status_code == 200:
        # json
        df_sam = pd.DataFrame(sam_response.json()['data'])
        # extract needed metadata
        df_sam['desc'] = df_sam['attributes'].apply(lambda x: x['sample-desc'])
        df_sam['sam_source'] = df_sam['desc'].str.split('.', expand=True)[0]
        df_sam['sam_read'] = df_sam['desc'].str.split('.', expand=True)[2]
    else: 
        raise Exception(f"Failed to download {url_sam}")
        print(sam_response)
    ana_response = requests.get(url_ana)
    if ana_response.status_code == 200:
        # json
        df_ana = pd.DataFrame(ana_response.json()['data'])
        # extract needed metadata
        df_ana['run_id'] = df_ana['relationships'].apply(lambda x: x['run']['data']['id'])
        df_ana['sample_id'] = df_ana['relationships'].apply(lambda x: x['sample']['data']['id'])
    else: 
        raise Exception(f"Failed to download {url_ana}")
        print(ana_response)
    # merge
    metadata[accession] = df_ana.merge(df_sam[['id', 'desc', 'sam_source', 'sam_read']], left_on='sample_id', right_on='id', how='left')


Processing MGYS00005058
Processing MGYS00005056


In [132]:
enf_all = metadata[enf_accession].merge(enf_go, left_on='run_id', right_on='sample', how='inner')
inf_all = metadata[inf_accession].merge(inf_go, left_on='run_id', right_on='sample', how='inner')
# sanity check
enf_all.shape, inf_all.shape

((24, 13), (24, 13))

In [136]:
# pairing 
df_data = enf_all.merge(
    inf_all,
    on=['sam_source', 'sam_read'], suffixes=('_enf', '_inf')
)[['run_id_enf', 'run_id_inf', 'sam_source', 'sam_read', 'abundance_enf', 'abundance_inf']]

df_data

,run_id_enf,run_id_inf,sam_source,sam_read,abundance_enf,abundance_inf
0,ERR2985255,ERR2814663,TG,READ2 Taxonomy ID:256318,3.257,4.227
1,ERR2985256,ERR2814664,MN,READ2 Taxonomy ID:256318,2.573,3.847
2,ERR2985257,ERR2814651,AH,READ1 Taxonomy ID:256318,4.299,4.087
3,ERR2985258,ERR2814667,TE,READ1 Taxonomy ID:256318,2.759,3.437
4,ERR2985259,ERR2814660,FD,READ1 Taxonomy ID:256318,3.365,3.487
5,ERR2985260,ERR2814652,AD,READ1 Taxonomy ID:256318,3.123,3.196
6,ERR2985261,ERR2814655,EM,READ2 Taxonomy ID:256318,2.811,3.680
7,ERR2985262,ERR2814647,MN,READ1 Taxonomy ID:256318,2.648,3.854
8,ERR2985263,ERR2814649,ZR,READ2 Taxonomy ID:256318,3.575,3.899
9,ERR2985264,ERR2814646,FD,READ2 Taxonomy ID:256318,3.193,3.451


In [134]:
from acore.permutation_test import paired_permutation

# optional for reproducibility
import numpy as np
rng = np.random.default_rng(12345)

In [135]:
for metric in ['t-statistic', 'mean', np.mean]:
    result = paired_permutation(
        df_data['abundance_inf'].to_numpy(),
        df_data['abundance_enf'].to_numpy(), 
        metric=metric, 
        n_permutations=10000, 
        rng=rng
    )

    print(result)

{'metric': <function ttest_rel at 0x115df23e0>, 'observed': TtestResult(statistic=np.float64(6.7389860601792275), pvalue=np.float64(7.122287781830209e-07), df=np.int64(23)), 'p_value': np.float64(0.0)}
{'metric': <function mean at 0x130d17df0>, 'observed': np.float64(0.5350826397500547), 'p_value': np.float64(0.0)}
{'metric': <function mean at 0x130d17df0>, 'observed': np.float64(0.5350826397500547), 'p_value': np.float64(0.0)}
